In [359]:
import  pytesseract
from PIL import Image
import cv2
from matplotlib import pyplot as plt
import os
import numpy as np

#import easyocr as ocr
#C:\Users\RAYYAN TECH\AppData\Local\Programs\Tesseract-OCR

In [360]:
pytesseract.pytesseract.tesseract_cmd=r"C:\\Users\\RAYYAN TECH\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe"

In [361]:
def convert_to_png(path):
    print("converting the image type")
    image = Image.open(path)
    png_path = f'Converted_image.png'
    try: 
        while True: 
            image.seek(image.tell() + 1) 
        
    except EOFError: 
        pass

    image.seek(0) 
    image.save(png_path, 'PNG')
    return png_path


In [362]:
def addWeight(image):
    brightness = 10 
    # Adjusts the contrast by scaling the pixel values by 2.3 
    contrast = 1.3  
    weighted = cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness) 
    return weighted

In [363]:
def processCarPlate(car_plate):
    blur = cv2.bilateralFilter(car_plate,9, 95, 95)
    weighted = addWeight(blur)
    canny = cv2.Canny(weighted,155, 255)
    binary = cv2.adaptiveThreshold(canny,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,75,2)
    return weighted

In [364]:

def cropImage(image, loc):
    location = loc
    rect = cv2.minAreaRect(location)
    box = cv2.boxPoints(rect)
    box = np.int64(box)

    mask = np.zeros(image.shape, dtype='uint8')
    cv2.drawContours(mask,[box],0,255,-1)

    new_image = cv2.bitwise_and(image,image,mask=mask)
    cv2.imshow('new Image', new_image)

    (x,y) = np.where(mask==255)
    (x1,y1) = (np.min(x), np.min(y))
    (x2,y2) = (np.max(x), np.max(y))

    ROI = image[x1:x2+1, y1:y2+1]
    return ROI


In [459]:
def preProcess(path):
    png_path = convert_to_png(path)
    car_plate = None

    img = cv2.imread(png_path)
    if img is None:
        print(f"Error: Image is not loaded correctly")
    else:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_img = cv2.resize(gray_img,(800,800),interpolation=cv2.INTER_LANCZOS4)

        blurred_img = cv2.GaussianBlur(gray_img,(1,1), cv2.BORDER_DEFAULT)
        #blurred_img = cv2.bilateralFilter(gray_img, 15, 75, 75)

        canny = cv2.Canny(blurred_img, 125, 255)
        cv2.imshow("Canny", canny)
        binary_img = cv2.adaptiveThreshold(canny, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,3,2)
        cv2.imshow("Binary image", binary_img)
        contours,_ = cv2.findContours(binary_img,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        #cv2.drawContours(gray_img,contours,-1,255,2)
        #cv2.imshow("Gray", gray_img)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:20]
        location = None
        for contour in contours:
            approx = cv2.approxPolyDP(contour,20, True)
            if len(approx) == 4:
                location = approx
                break

        if location is None:
            print("No number contour is found.")
        else:
            ROI = cropImage(gray_img, location)
            car_plate = ROI

        

        
    
        #cv2.imshow("Number Plate", ROI)
        cv2.imwrite("preprocessed_image.png", car_plate)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        

    return car_plate





In [457]:


def extractText(proc_image):
    proc_car_plate = processCarPlate(proc_image)
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(proc_car_plate, config=custom_config)
    return text



In [458]:
path ='data/new-images/image-04.jpg'
pro_image = preProcess(path)
text = extractText(proc_image=pro_image)
print(text)


converting the image type
FO eee



In [434]:
cv2.destroyAllWindows()